# Task 1

### Transaction Definition: (板橋測站的溫度, 北區用電量) for each hour

Run the following query, store the results in TSV format.  
Note: Round `DATETIME` towards its nearest hour. (e.g. 2017/10/16 14:37:52 -> 2017/10/16 15:00:00)

```sql
SELECT DATE_FORMAT(DATE_ADD(Power.updateTime, INTERVAL 30 MINUTE), '%Y-%m-%d %H:00:00') AS time,
       逐時觀測.溫度,
       Power.northUsage
FROM Power
INNER JOIN 逐時觀測
ON DATE_FORMAT(DATE_ADD(Power.updateTime, INTERVAL 30 MINUTE), '%Y-%m-%d %H:00:00')
   = DATE_FORMAT(DATE_ADD(逐時觀測.時間, INTERVAL 30 MINUTE), '%Y-%m-%d %H:00:00')
WHERE 逐時觀測.測站 = 'BANQIAO,板橋'
```

### Discretization: 溫度四捨五入至個位，用電量四捨五入至十位

將用電量加上 10000，以便區分溫度與用電量。  
Note: 實際上不會剛好是「四捨五入」，因為浮點數在電腦裡是用二進制來儲存的，不過這並不影響結果。

In [1]:
dataset = []
with open('task1.tsv') as file:
    for line in file:
        fields = line.split('\t')
        assert len(fields) == 3
        t = int(round(float(fields[1])))
        p = int(round(float(fields[2]), -1)) + 10000
        dataset.append([t, p])

In [2]:
len(dataset)

4636

In [3]:
print(dataset[0:10])

[[27, 10840], [26, 10830], [26, 10790], [26, 10770], [27, 10780], [27, 10780], [27, 10810], [27, 10810], [28, 10810], [29, 10800]]


### Algorithm: FP-Growth

In [4]:
from orangecontrib.associate.fpgrowth import *
itemsets = dict(frequent_itemsets(dataset, 0.01))

In [5]:
len(itemsets)

70

In [9]:
rules = association_rules(itemsets, 0.1)

### Rules Discovered: None

Cannot find any rules. Need coarser granularity.

In [10]:
for antecedent, consequent, support, confidence in rules:
    print(antecedent, '->', consequent, 'supp =', support, 'conf =', confidence)

### Discretization: 溫度四捨五入至個位，以五度為一單位，用電量四捨五入至十位，以五十萬瓩為一單位

In [1]:
dataset = []
with open('task1.tsv') as file:
    for line in file:
        fields = line.split('\t')
        assert len(fields) == 3
        t = int(round(float(fields[1])))
        p = int(round(float(fields[2]), -1)) + 10000
        dataset.append([t // 5, p // 50])

### Algorithm: FP-Growth

In [13]:
!echo 666

666
